### IMPORTS

#### Search Wikidata

In [ ]:
from qwikidata.sparql import (get_subclasses_of_item, return_sparql_query_results)
import pandas as pd

#### Beautiful Soup Scrape for Tables

In [ ]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from urllib.request import urlopen
import re
import random

from itertools import chain
import json

###  Functions

In [ ]:
# Put wikidata results into a dataframe
def niceify(results):
    count = 0 
    noice = []
    for res in results['results']['bindings']:
        temp={}
        for h in headers:
            if h in res.keys():
                temp[h] = res[h].get('value', "None")
        
            noice.append(temp)
    df = pd.DataFrame(noice)
    
    return df

In [ ]:
# Delete any duplicate rows and delete rows without a URL
def de_dupe(df, NaN_column='article'):
    
    df = df.dropna(subset=[NaN_column])
    df = df.drop_duplicates()
    df = df.reset_index(drop=True, inplace=False)
    
    return df

In [ ]:
#List of dicts; dict values are of dataframes of the tables for that particular url
def table_scrapes(url_list, prob=.05, maxdelay=3):
    
    print("Starting your scrape...")
    
    #Add key to tuple in "amplified" list of urls from wikidata search
    urls_amp = [(x, x.split("/")[-1]) for x in url_list]
    
    super_list_dict = []
    count = 0
    success = 0
    tot = len(url_list)
    delay = [decision(prob,maxdelay) for x in range(tot)]
    
    # Iterate over each url, pull out any wikitables and throw into super list with key=article url tag
    for url_tup in urls_amp:

        if delay[count] != 0:
            print(f'Pause scrape for {delay[count]} second(s)')
            time.sleep(delay[count])
        
        # unique keys for the url
        url = url_tup[0]
        key = url_tup[1]

        #soup it 
        html = urlopen(url)
        soup = BeautifulSoup(html, 'html.parser')

        # Filter table search to just wikitables which will clean out junk tables
        #tables = soup.find_all("table",{"class":"wikitable"})
        tables = soup.find_all("table", {"class":["wikitable", "infobox vevent"]})
        
        if tables != []:
            try:
                # build temp dicts to dump into temp list that dumps into overall super list 
                df_dict = {}
                temp_list = []

                for table in tables:

                    str_tab = str(table)

                    #one-off replace to fix html error
                    str_tab = str_tab.replace('6;', '6')

                    temp_df = pd.read_html(str_tab)[0]
                    temp_list.append(temp_df)

                df_dict[key] = temp_list
                df_dict["url"] = url
                
                # add table df to super list
                super_list_dict.append(df_dict) 
                
                #success counter
                success +=1

            except:
                print(f"Skipping Parsing Error for: {url}")
        else:
            print(f"No wikitables found, skipping: {url}")
        
        #Display
        if count%100 == 50:
            print(f'**** Scraped {count} of {tot} urls ****')
        count += 1    
    
    print("!!!! Complete !!!!")       
    print(f'{success} of {len(url_list)} urls have wikitables')    

    return super_list_dict 

In [ ]:
def lister(soup):
    result = []
    uls = soup.find_all('ul', {'class': 'mw-allpages-chunk'})
    base ="https://en.wikipedia.org"
    for ul in uls:
        for li in ul.find_all('li'):
            for link in li.find_all('a'):
                url = link.get('href')
                contents = link.text
                #print(f'{base}{url} Key: {contents}')
                temp = base+url
                result.append(temp)
    return result

In [ ]:
def get_all_urls(urls):
    res = []
    for url in urls:
        html = urlopen(url)
        soup = BeautifulSoup(html, 'html.parser')

        temp = lister(soup)
        res.append(temp)

   # res = res[0] 
    return res

In [ ]:
def display_all_results(super_list_dict):

    df = pd.DataFrame(columns=["Number of Tables", "key", "url"])
    i = 0
    
    for d_ in super_list_dict:
        key = [x for x in d_.keys()][0]
        url = d_.get('url', "None")
        num_df = len(d_[key])
        df.loc[i] = [str(num_df), key, url]
        i += 1
            
    #print(f' num df: {num_df} | key: {key} | url: {url}')

    return df 

In [ ]:
#random delay
def decision(probability, maxdelay=8):
    if random.random() < probability:
        delay = random.randint(1, maxdelay)
    else:
        delay = 0
    return delay

In [ ]:
# Search across all text for keyword
def keyword_search(soup_results, keyword):
    
    search_result = []
    outer_cnt = 0 
    
    for res in soup_results:

        for k,v in res.items():
            if k != "url":
                key=k
            if k == "url":
                url=res[k]

        inner_cnt = 0
        for frame in res[key]:

            t=frame.to_dict()
            values = set(chain.from_iterable(i.values() for i in t.values()))
            values = (str(values).split())

            temp_result = []
            for elem in values:
                if keyword.lower() in elem.lower():

                    temp_result = [outer_cnt, inner_cnt, key, url]

            if temp_result != []:
                search_result.append(temp_result)

            inner_cnt += 1    
        outer_cnt += 1
        
    for line in search_result:
        print(line)
    
    if search_result == []:
        return print("No results found")
        
    return search_result

In [ ]:
# IN PROGRESS...
def search_title1(keyword, df):
    
    print(df[df['key'].str.contains(keyword, case=False)])      

In [ ]:
# IN PROGRESS...
def search_title2(keyword, url_list):
    
    search_words = [x.split("/")[-1].lower() for x in url_list]

    index = []
    for words in search_word:
        temp_index = []
        if keyword.lower() in words:
            ind = search_words.index(words)
            index.append(ind)
            
    return index

### Wikidata Search

#### Get list of Ethiopian urls to scrape for tables

In [ ]:
# Create list of column headers you want (reference the "SELECT" line in Query assignment)
headers = ['cid','cidLabel','article']

In [ ]:
# Generate SPARQL Query 
sparql_query = """
PREFIX schema: <http://schema.org/>
PREFIX wikibase: <http://wikiba.se/ontology#>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>

SELECT ?cid ?country ?article WHERE {

   #?cid wdt:P31 wd:Q3839081 .
   
   ?cid  wdt:P17 wd:Q115 .
   
   OPTIONAL {
      ?cid rdfs:label ?country filter (lang(?country) = "en") .
    }
    OPTIONAL {
      ?article schema:about ?cid .
      ?article schema:inLanguage "en" .
      ?article schema:isPartOf <https://en.wikipedia.org/> .
      #?article schema:isPartOf <https://en.wikipedia.org/ .
    }
} 
"""

results = return_sparql_query_results(sparql_query)

In [ ]:
# Transform wikidata search results to list of urls to scrape for tables
df = de_dupe(niceify(results), 'article')
df_urls = df['article']
wikidata_urls = list(df_urls)
wikidata_urls[:5] + ["etc....."]

#### Scrape wikipedia for Ethiopian pages: https://en.wikipedia.org/wiki/Special:AllPages?from=ethiopia&to=&namespace=0

In [ ]:
# URLs below are from a manual search from the link above, the 3 pages have ETH-related pages
url1 = 'https://en.wikipedia.org/w/index.php?title=Special:AllPages&from=Ethiopia'
url2 = 'https://en.wikipedia.org/w/index.php?title=Special:AllPages&from=Ethiopian+Civil+Aviation+Agency'
url3 = 'https://en.wikipedia.org/w/index.php?title=Special:AllPages&from=Ethiopian+farming'

urls=[url1, url2,url3]

wikipedia_urls = get_all_urls(urls)
wikipedia_urls = [i for g in wikipedia_urls for i in g]

In [ ]:
# Add the wikidata and wikipedia url lists together
url_list = wikidata_urls + wikipedia_urls

#Clean out trailing urls not associated with ETH
ind = url_list.index('https://en.wikipedia.org/wiki/Ethiopiochamus_centralis')

url_list = url_list[:ind]
len(url_list)

### beautiful soup to scrape list of wikipedia pages' urls and return any tables that are on the page

In [ ]:
soup_results = table_scrapes(url_list)

In [ ]:
# delta between urls scraped and urls with at least one wikitable
print(f'{len(soup_results)} of {len(url_list)} urls have wikitables')

In [ ]:
# Show results of urls WITH tables
pd.set_option('display.max_columns', None)
pd.set_option("max_colwidth", None)
pd.set_option("max_rows", None)
df = display_all_results(soup_results)
df.head()

In [ ]:
# Enter keyword to search for TABLE DATA:
keyword = "farm"

search = keyword_search(soup_results, keyword)

In [ ]:
soup_results[176]['Healthcare_in_Ethiopia'][0]

### TESTING...

In [ ]:
u= ['https://en.wikipedia.org/wiki/2021_Ethiopian_general_election']
tt= table_scrapes(u, prob=.05, maxdelay=3)

df2 = display_all_results(tt)
df2.head() 

In [ ]:
a = tt[0]['2021_Ethiopian_general_election'][0]
a